# "Hacker News" Posts Analysis

The goal of this project is to analyze two different types of posts on the popular technology site Hacker News [(Dataset here)](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts). 

The two types of posts are Ask HN and Show HN. Ask HN posts are submitted by users who want to ask a specific question to the Hacker News community, while Show HN posts are submitted to showcase a project, product, or something interesting. The aim is to compare these two types of posts and determine which one receives more comments on average and whether posts created at a certain time receive more comments on average. 

It's important to note that the dataset used in this analysis was reduced to 20,000 rows from an initial 300,000 rows by eliminating submissions that received no comments and randomly sampling from the remaining submissions.


## Introduction

Importing Data and Removing Headers to understand the dataset.

In [34]:
import csv # to save memory, you can also import only reader: from csv import reader

workfile = open("hacker_news.csv")
hn = list(csv.reader(workfile))
hn_header = hn[0]    # Header only
hn = hn[1:]          # Data only (remove last 2 lines if dataset has no header)

print(hn_header)     # From here on, we print with spacing for better clarity
print('\n')

for i in range(3):    # Only first 3 rows
    print(hn[i])
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']




## Extracting "Ask HN" and "Show HN" Posts

First, we'll identify posts that begin with either "Ask HN" or "Show HN" and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.


In [13]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1].lower()   # Starts / Ends with are case sensitive, so I lower-case the data being check
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [30]:
total_ask_comments = 0

for row in ask_posts:
    comment = int(row[4])
    total_ask_comments += comment

avg_ask_comments = total_ask_comments / len(ask_posts)

print("Average ask comments:  {avg:.2f}".format(avg = avg_ask_comments))


Average ask comments:  14.04


In [31]:
total_show_comments = 0

for row in show_posts:
    comment = int(row[4])
    total_show_comments += comment

avg_show_comments = total_show_comments / len(show_posts)

print("Average show comments:  {avg:.2f}".format(avg = avg_show_comments))

Average show comments:  10.32


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.



## Finding the Amount of Ask Posts and Comments by Hour Created


Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.



In [49]:
import datetime as dt

result_list = []

for row in ask_posts:
    result_list.append([row[6], int(row[4])])  # A dictionary containing "Creation Time" & "Comments #"
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    date = row[0]
    date = dt.datetime.strptime(date, "%m/%d/%Y %H:%M") 
    hour = date.strftime("%H")
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

## Calculating the Average Number of Comments for Ask HN Posts by Hour


In [55]:
avg_by_hour = []  # Average comments per hour = tot comments / tot post counts

for hour in comments_by_hour:
    comments = comments_by_hour[hour] 
    avg = (comments / counts_by_hour[hour])
    avg_by_hour.append([hour, float(f"{avg:.2f}")])   # Formatting average, and converting it back into number

avg_by_hour

[['09', 5.58],
 ['13', 14.74],
 ['10', 13.44],
 ['14', 13.23],
 ['16', 16.8],
 ['23', 7.99],
 ['12', 9.41],
 ['17', 11.46],
 ['15', 38.59],
 ['21', 16.01],
 ['20', 21.52],
 ['02', 23.81],
 ['18', 13.2],
 ['03', 7.8],
 ['05', 10.09],
 ['19', 10.8],
 ['01', 11.38],
 ['22', 6.75],
 ['08', 10.25],
 ['04', 7.17],
 ['00', 8.13],
 ['06', 9.02],
 ['07', 7.85],
 ['11', 11.05]]

## Sorting and Printing Values from a List of Lists


In [62]:
swap_avg_by_hour = []       # Create a list with average in 1st column, hour in 2nd

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

[[5.58, '09'], [14.74, '13'], [13.44, '10'], [13.23, '14'], [16.8, '16'], [7.99, '23'], [9.41, '12'], [11.46, '17'], [38.59, '15'], [16.01, '21'], [21.52, '20'], [23.81, '02'], [13.2, '18'], [7.8, '03'], [10.09, '05'], [10.8, '19'], [11.38, '01'], [6.75, '22'], [10.25, '08'], [7.17, '04'], [8.13, '00'], [9.02, '06'], [7.85, '07'], [11.05, '11']]


In [65]:
swap_avg_by_hour.sort(reverse=True) # Ordering by highest average comment #

print(swap_avg_by_hour)

[[38.59, '15'], [23.81, '02'], [21.52, '20'], [16.8, '16'], [16.01, '21'], [14.74, '13'], [13.44, '10'], [13.23, '14'], [13.2, '18'], [11.46, '17'], [11.38, '01'], [11.05, '11'], [10.8, '19'], [10.25, '08'], [10.09, '05'], [9.41, '12'], [9.02, '06'], [8.13, '00'], [7.99, '23'], [7.85, '07'], [7.8, '03'], [7.17, '04'], [6.75, '22'], [5.58, '09']]


In [92]:
print("Top 5 Hours for Ask Posts Comments")    

for row in swap_avg_by_hour[:5]:
    
    avg = row[0]
    hour = row[1]       
    hour = dt.datetime.strptime(hour, "%H").strftime("%H:%M")  # Converting from 20 to 20:00 
    
    print("{h} : {a} average comments per post".format(h=hour, a=avg))

Top 5 Hours for Ask Posts Comments
15:00 : 38.59 average comments per post
02:00 : 23.81 average comments per post
20:00 : 21.52 average comments per post
16:00 : 16.8 average comments per post
21:00 : 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

## Conclusion

In this study, we examined ask and show posts to identify the type of post and the time that typically receive the highest number of comments. Our findings suggest that to increase the comment count, it is advisable to classify the post as an ask post and publish it between 3:00 pm and 4:00 pm EST.

However, it is important to note that the dataset we analyzed excluded posts that did not receive any comments. Therefore, a more precise statement would be that among the posts that did receive comments, ask posts tended to receive more comments on average, and ask posts created between 3:00 pm and 4:00 pm EST received the highest number of comments on average.